In [1]:
import pandas as pd

In [17]:
# Read in data
# products = pd.read_csv("Resources/products.csv")
# accounts = pd.read_csv("Resources/accounts.csv")
ast = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("Resources/group_product_associations.csv")
# groups = pd.read_csv("Resources/groups.csv")
# payments = pd.read_csv("Resources/payments.csv")

In [18]:
ast.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

In [ ]:
# State as of: date (Using 'started_when' for now)
# total time in state as of date: -- time in state as of date until changed? or time since date moving forward?

In [ ]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

In [ ]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

In [ ]:
# Change data type from string to datetime object
payments['effective_when'] = pd.to_datetime(payments['effective_when'])

In [ ]:
# Create limited dataframe starting at indicated timestamp 
payments_date_limited = payments.loc[payments['effective_when'] >= timestamp_dt]

## 3) Devise a method of generating a table of all Accounts with their cumulative payments to date as of any arbitrary timestamp in the past.

In [3]:
# Set currency
currency = payments.currency[0]

In [10]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-01-01


In [11]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

2020-01-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [12]:
# Change data type from string to datetime object
payments['effective_when'] = pd.to_datetime(payments['effective_when'])

In [13]:
# Create limited dataframe starting at indicated timestamp 
payments_date_limited = payments.loc[payments['effective_when'] >= timestamp_dt]

,id,effective_when,account_id,currency,amount
12,13,2020-01-05 09:25:19.571392,2,KES,368
13,14,2020-01-12 11:43:07.571392,2,KES,380
14,15,2020-01-19 14:23:11.571392,2,KES,422
15,16,2020-01-27 02:37:47.571392,2,KES,388
16,17,2020-02-03 20:07:42.571392,2,KES,422


In [14]:
# Group by account_id and count number of records
num_payments = payments_date_limited.groupby('account_id').count()

# Flatten dataframe
num_payments = num_payments.rename_axis(None, axis=1).reset_index()

# Drop columns
num_payments = num_payments.drop(columns=['effective_when','currency','amount'])

# Rename column
cumulative_payments = num_payments.rename(columns={"id":"total_num_payments"})

,account_id,total_num_payments
0,2,8
1,3,7
2,4,8
3,5,5
4,6,8


In [15]:
# Group by account_id and sum payment amounts
sum_payments = payments_date_limited.groupby('account_id').sum()

# Flatten dataframe
sum_payments = sum_payments.rename_axis(None, axis=1).reset_index()

# Select one column
sum_payments = sum_payments['amount']

# Add series to cumulative_payments dataframe
cumulative_payments[f'total_sum_payments_{currency}'] = sum_payments

print(f"CUMULATIVE PAYMENTS BY ACCOUNT SINCE: {timestamp_str[:10]}")
cumulative_payments.head()

CUMULATIVE PAYMENTS BY ACCOUNT SINCE: 2020-01-01


,account_id,total_num_payments,total_sum_payments_KES
0,2,8,3102
1,3,7,3213
2,4,8,4599
3,5,5,2386
4,6,8,2360


In [23]:
# Export as csv file
# cumulative_payments.to_csv(f"Generated_Tables/cumulative_payments_by_account_since_{timestamp_str[:10]}.csv", index=False)